In [1]:
import pytesseract
from PIL import Image
from PIL import ImageFile
import pandas as pd
import re
import numpy as np
import warnings
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

ImageFile.LOAD_TRUNCATED_IMAGES = True
pytesseract.pytesseract.tesseract_cmd = R'C:\Program Files\Tesseract-OCR\tesseract'
warnings.filterwarnings('ignore')

## 기존 정보 및 사전 작업

### 기존 데이터

In [2]:
df=pd.read_csv('./data/olive0_classify_allergy_gluten.csv',encoding='utf-8')
df

,brand,product,text,cate,animal,allergy,gluten
0,랩시리즈,랩시리즈 올인원 트리트먼트 50ml 1+1 기획세트,"사이클로덱스트린,포타슘스테아레이트,포도씨추출물,스쿠알란,옥틸도데실네오펜타노에이트,에...",스킨케어,1,발견사항 없음,발견사항 없음
1,유세린,유세린 하이알루론 아이크림X나이트크림 기획(아이크림 15ml+나이트크림 50ml+컨...,"에칠헥실리세린,부틸렌라이콜디카프릴레이트,이소프로필팔미테이트,하이드로제네이티드코코리세...",스킨케어,1,발견사항 없음,발견사항 없음
2,아이디얼포맨,아이디얼 포맨 퍼펙트스킨케어 2종세트(미니어처 3종 증정),"페닐알라닌,무화과추출물,홀리바질잎추출물,나이아신아마이드,카프릴릭,다이소듐이디티에이,...",스킨케어,1,"부틸페닐메틸프로피오날,리모넨,리날룰",발견사항 없음
3,다슈,다슈 맨즈 아쿠아 토너/로션 153ml 2종 세트(+토너&로션&클렌징 30ml증정),"개청각추출물,연꽃씨추출물,바실러스,마트리카리아꽃추출물,띠뿌리발효추출물,페닐알라닌,민...",스킨케어,1,"부틸페닐메틸프로피오날,리모넨,리날룰",발견사항 없음
4,달바,달바 화이트 트러플 퍼스트 아로마틱 토너 155ml,"네오펜틸글라이콜다이헵타노에이트,하이드록시에틸우레아,돌콩오일,브로멜라인,소듐클로라이드...",스킨케어,1,"리날룰,리모넨,헥실신남알",발견사항 없음
...,...,...,...,...,...,...,...
3748,데싱디바,데싱디바 매직프레스 베이스 쉴드,"토코페롤,스위트아몬드오일,아이소프로필알코올,해바라기씨오일,포스포릭애씨드,실리카,트라...",네일,0,발견사항 없음,발견사항 없음
3749,엣지유,엣지유 샵케어 탑젤 글로시&매트,"유칼립투스잎추출물,하이드록시사이클로헥실페닐케톤,셀룰로오스아세테이트부티레이트,헤마,매...",네일,0,발견사항 없음,발견사항 없음
3750,데싱디바,데싱디바 리치 세럼,"벤질살리실레이트,포도씨오일,미네랄오일,스타이렌코폴리머,부틸렌,에틸렌,부틸페닐메틸프로...",네일,0,"벤질살리실레이트,부틸페닐메틸프로피오날,헥실신남알",발견사항 없음
3751,위드샨,위드샨 에코 네일 리무버 200ml,"제라니올,리날룰,프로필렌글라이콜,정제수,라벤더오일,토코페릴아세테이트,오레가노오일,로...",네일,0,"제라니올,리날룰,리모넨",발견사항 없음


### 분류 기준표

In [3]:
with open('./list/check_lst.txt','r') as file:
    lst=file.readlines()

check_lst=[]
for word in lst:
    word=word.replace('\n','')
    word=word.replace(' ','')
    if len(word)>0:
        check_lst.append(word)

### 추천시스템 함수

In [4]:
def find_sim_text(df,sorted_ind,product_name,top_n,vegan=0,cate=0):

    title_product=df[df['product']==product_name]
    title_index=title_product.index.values

    similar_indexes=sorted_ind[title_index]
    similar_indexes=similar_indexes[similar_indexes!=title_index]

    similar_indexes=similar_indexes.reshape(-1)
    indexes=similar_indexes

    # 비건 옵션
    if vegan==1:
        vegan_indexes=df.loc[df['animal']==0].index.values

    # 카테고리 옵션
    if cate==1:
        category=df.loc[df['product']==product_name,'cate'].values
        cate_indexes=df.loc[df['cate']==category[0]].index.values
        
    # 옵션 적용
    option_lst=[]
    if vegan==1 and cate==1: # 비건, 카테고리
        for idx in similar_indexes:
            if idx in vegan_indexes and idx in cate_indexes:
                option_lst.append(idx)
        indexes=option_lst
    
    elif vegan==1 and cate==0: # 비건
        for idx in similar_indexes:
            if idx in vegan_indexes:
                option_lst.append(idx)
        indexes=option_lst

    elif vegan==0 and cate==1: # 카테고리
        for idx in similar_indexes:
            if idx in cate_indexes:
                option_lst.append(idx)
        indexes=option_lst
        
    return df.iloc[indexes][:top_n]

In [5]:
df['text']=df['text'].apply(lambda x:x.replace(',',' '))

### 알레르기 및 글루텐 정보

In [6]:
cosmetic_allergy_lst=['아밀신남알', '벤질알코올', '신나밀알코올', '시트랄', '유제놀', '하이드록시시트로넬알', '이소유제놀', '아밀신나밀알코올', '벤질살리실레이트', '신남알', '쿠마린', '제라니올', '아니스에탄올', '벤질신나메이트', '파네솔', '부틸페닐메틸프로피오날', '리날룰', '벤질벤조에이트', '시트로넬롤', '헥실신남알', '리모넨', '메칠2옥티노에이트', '알파이소메칠이오논', '참나무이끼추출물', '나무이끼추출물']
glu=['글루테닌','글리아딘','보리','귀리','밀','강력분','중력분','박력분','밀가루']

## 새로운 데이터 입력

### 1) 화장품 정보 입력 및 성분 사진 인식

In [7]:
print('화장품 정보를 입력하세요')
brand=[] ; product=[]; category=[]

brand.append(input(print('1) 화장품 브랜드:')))
print('=>',brand[0])

product.append(input(print('2) 화장품 이름:')))
print('=>',product[0])

category.append(input(print('3) 화장품 종류:')))

print('=>',category[0])

화장품 정보를 입력하세요
1) 화장품 브랜드:
=> 닥터지
2) 화장품 이름:
=> 레드 블레미쉬 모이스처 클렌징 폼
3) 화장품 종류:
=> 스킨케어


In [8]:
print('화장품 성분 사진을 불러옵니다.')

화장품 성분 사진을 불러옵니다.


In [9]:
img=Image.open('./img/화장품_성분만.jpg')
result=pytesseract.image_to_string(img,lang='kor')

In [10]:
result

'[전성분] 정제수, 스테아릭애씨드, 글리세린, 라우릭애씨드,\n포타슴하이드록사이드, 미리스틱애씨드, 글리세릴스테\n아레이트, 피이지-100스테아레이트, 프로필렌글라이콜,\n코카미도프로필베타인, 비즈왁스, 소듬클로라이드, 폴리\n소르베이트20, 디소듬이디티에이, 하이드록시에칠셀룰\n로오스, 부틸렌글라이콜, 쌀겨추출물, 파파야열매추출물,\n접시꽃추출물, 알로에베라잎추출물, 왕글낄추출물,\n은행잎추출물, 레몬추출물, 소듬피씨에이, 페녹시에탄올,\n에칠핵실글리세린, 향료, 부틸페닐메칠프로피오날, 리날룰,\n렉실신남알\n'

In [11]:
start_idx=re.search('전성분',result).end()
#end_idx=re.search('피부 자극',result).start()
ingredient=result[start_idx:]

In [12]:
ingredient=ingredient.replace('.',',')
ingredient=ingredient.replace(' ',',')
ingredient=re.sub('[^가-힣,]','',ingredient)
ingredient=re.sub(',+',',',ingredient)
ingredient

',정제수,스테아릭애씨드,글리세린,라우릭애씨드,포타슴하이드록사이드,미리스틱애씨드,글리세릴스테아레이트,피이지스테아레이트,프로필렌글라이콜,코카미도프로필베타인,비즈왁스,소듬클로라이드,폴리소르베이트,디소듬이디티에이,하이드록시에칠셀룰로오스,부틸렌글라이콜,쌀겨추출물,파파야열매추출물,접시꽃추출물,알로에베라잎추출물,왕글낄추출물,은행잎추출물,레몬추출물,소듬피씨에이,페녹시에탄올,에칠핵실글리세린,향료,부틸페닐메칠프로피오날,리날룰,렉실신남알'

In [13]:
input_data=pd.DataFrame({'brand':brand,'product':product,'text':ingredient,'cate':category})
input_data

,brand,product,text,cate
0,닥터지,레드 블레미쉬 모이스처 클렌징 폼,",정제수,스테아릭애씨드,글리세린,라우릭애씨드,포타슴하이드록사이드,미리스틱애씨드,글리...",스킨케어


### 2) 동물성/식물성 화장품 분류

In [14]:
flag=0
input_data['animal']='-'
for word in input_data['text'][0].split(','):
    if len(word)>0 and word in check_lst:
        flag=1
        break
if flag==1:
    print('['+input_data['product'][0]+'] 에는 동물성 성분이 포함되어 있어요.')
    input_data['animal'][0]='1'
else:
    print('['+input_data['product'][0]+'] 에는 동물성 성분이 발견되지 않았어요.')
    input_data['animal'][0]='0'

[레드 블레미쉬 모이스처 클렌징 폼] 에는 동물성 성분이 포함되어 있어요.


### 3) 알레르기 확인

In [20]:
allergy_lst=[]
for word in input_data['text'][0].split(' '):
    if word in cosmetic_allergy_lst:
        allergy_lst.append(word)
if len(allergy_lst)>0:
    print('['+input_data['product'][0]+']에는 알레르기 성분 '+str(allergy_lst)+'이 들어있어요.')
else:
    print('['+input_data['product'][0]+']에는 알레르기 성분이 발견되지 않았어요')

[레드 블레미쉬 모이스처 클렌징 폼]에는 알레르기 성분 ['리날룰']이 들어있어요.


### 4) 글루텐 확인

In [21]:
flag=0
for word in input_data['text'][0].split(' '):
    if word in glu:
        flag=1
        break
if flag==1:
    print('['+input_data['product'][0]+']에는 글루텐이 포함되어 있습니다.')
else:
    print('['+input_data['product'][0]+']에는 글루텐이 포함되어 있지 않습니다.')

[레드 블레미쉬 모이스처 클렌징 폼]에는 글루텐이 포함되어 있지 않습니다.


### 5) 추천시스템

In [22]:
input_data['text']=input_data['text'].apply(lambda x:x.replace(',',' '))

In [23]:
# 옵션 입력
print('카테고리 옵션을 선택하세요')
print('1) 같은 카테고리 상품만 궁금해요')
print('2) 다른 카테고리 상품도 궁금해요')
cate_option=input('1 / 2')
print('=> '+cate_option+'번 옵션 선택')

if cate_option=='1':
    cate_option=1
else:
    cate_option=0

print('\n비건 옵션을 선택하세요')
print('1) 동물성 성분이 없는 제품만 궁금해요')
print('2) 동물성 상관없이 모두 궁금해요')
vegan_option=input('1 / 2')

print('=> '+vegan_option+'번 옵션 선택')
if vegan_option=='1':
    vegan_option=1
else:
    vegan_option=0

카테고리 옵션을 선택하세요
1) 같은 카테고리 상품만 궁금해요
2) 다른 카테고리 상품도 궁금해요
=> 1번 옵션 선택

비건 옵션을 선택하세요
1) 동물성 성분이 없는 제품만 궁금해요
2) 동물성 상관없이 모두 궁금해요
=> 1번 옵션 선택


In [24]:
recom_df=pd.concat([input_data,df]).reset_index(drop=True)
count_vect=CountVectorizer(min_df=0,ngram_range=(1,2))
text_mat=count_vect.fit_transform(recom_df['text'])
text_sim=cosine_similarity(text_mat,text_mat)
text_sim_sorted_ind=text_sim.argsort()[:,::-1]

print('입력한 상품과 비슷한 제품 5가지를 보여드려요')
find_sim_text(recom_df,text_sim_sorted_ind,input_data['product'].values[0],10,vegan_option,cate_option)

입력한 상품과 비슷한 제품 5가지를 보여드려요


,brand,product,text,cate,animal,allergy,gluten
1325,참존,참존징코 내츄럴 클렌징폼 더블 기획 (180ml+180ml),포타슘하이드록사이드 레몬그라스추출물 라우릭애씨드 페퍼민트잎추출물 글리세린 리모넨 리...,스킨케어,0,"리모넨,리날룰",발견사항 없음
1290,라로슈포제,라로슈포제 똘러리앙 퓨리파잉 포밍 크림,미리스틱애씨드 테트라소듐이디티에이 팔미틱애씨드 포타슘하이드록사이드 정제수 라우릭애씨...,스킨케어,0,발견사항 없음,발견사항 없음
1264,라로슈포제,라로슈포제 똘러리앙 포밍클렌저 더블 기획,미리스틱애씨드 테트라소듐이디티에이 팔미틱애씨드 포타슘하이드록사이드 정제수 라우릭애씨...,스킨케어,0,발견사항 없음,발견사항 없음
1276,스트라이덱스,스트라이덱스 약알칼리성 바하 폼클렌저 기획 (150ml+15ml),포타슘하이드록사이드 소듐클로라이드 무화과추출물 미역추출물 에틸멘탄카복사마이드 라우릭...,스킨케어,0,"리모넨,리날룰",발견사항 없음
1262,아크네스,아크네스 클리어&화이트 포밍워시 1+1 기획,시트릭애씨드 팔미틱애씨드 포타슘하이드록사이드 라우릭애씨드 에탄올 글리세린 다이포타슘...,스킨케어,0,"리모넨,리날룰",발견사항 없음
1279,한율,한율 어린쑥 진정 맑은 클렌징폼 기획세트 (폼120g + 수분진정크림15g),팔미틱애씨드 포타슘하이드록사이드 라우릭애씨드 글리세릴스테아레이트 글리세린 다이소듐이...,스킨케어,0,발견사항 없음,발견사항 없음
1307,라로슈포제,라로슈포제 똘러리앙 포밍클렌저 125ml 기획,미리스틱애씨드 테트라소듐이디티에이 증정똘러리앙 팔미틱애씨드 포타슘하이드록사이드 정제...,스킨케어,0,발견사항 없음,발견사항 없음
1336,스킨푸드,스킨푸드 블랙슈가 퍼펙트 버블폼 200ml,마카뿌리추출물 라우릴포스페이트 포타슘하이드록사이드 소듐클로라이드 라우릭애씨드 글리세...,스킨케어,0,"리모넨,시트랄",발견사항 없음
1273,브링그린,브링그린 티트리시카 트러블클렌징폼 더블기획(200ml+200ml),베타글루칸 팔미틱애씨드 포타슘하이드록사이드 라우릭애씨드 글리세릴스테아레이트 글리세린...,스킨케어,0,발견사항 없음,발견사항 없음
1304,뉴트로지나,뉴트로지나 딥클린 젠틀 포밍 클렌저 220g+50g 기획,다이소듐이디티에이 미리스틱애씨드 소듐코코일글라이시네이트 코카미도프로필베타인 정제수 ...,스킨케어,0,발견사항 없음,발견사항 없음
